In [1]:
import tensorflow as tf
from tensorflow.contrib.rnn import LSTMCell, MultiRNNCell
import numpy as np

Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
x_s = tf.placeholder(name='x_s', shape=(1, 100), dtype=tf.float32)
x_n = tf.placeholder(name='x_n', shape=(1, 100), dtype=tf.float32)

with tf.variable_scope('encoder'):
    encoder = MultiRNNCell([LSTMCell(96, name='encin', reuse=tf.AUTO_REUSE), LSTMCell(96, name='encout', reuse=tf.AUTO_REUSE)])
    z_s, _ = encoder(x_s, [(tf.zeros((1, 96)), tf.zeros((1, 96))), (tf.zeros((1, 96)), tf.zeros((1, 96)))])
    z_n, _ = encoder(x_n, [(tf.zeros((1, 96)), tf.zeros((1, 96))), (tf.zeros((1, 96)), tf.zeros((1, 96)))])


decoder_s = MultiRNNCell([
    LSTMCell(96, name='decinone', reuse=tf.AUTO_REUSE), 
    LSTMCell(96, name='decoutone', reuse=tf.AUTO_REUSE),
    LSTMCell(1000, name='vocaboutone', reuse=tf.AUTO_REUSE)])
decoder_n = MultiRNNCell([
    LSTMCell(96, name='decintwo', reuse=tf.AUTO_REUSE), 
    LSTMCell(96, name='decouttwo', reuse=tf.AUTO_REUSE),
    LSTMCell(1000, name='vocabouttwo', reuse=tf.AUTO_REUSE)])

xlogits_s,_= decoder_s(z_s, [(tf.zeros((1, 96)), tf.zeros((1, 96))), (tf.zeros((1, 96)), tf.zeros((1, 96))),(tf.zeros((1, 1000)), tf.zeros((1, 1000)))])
xlogits_n,_= decoder_n(z_n, [(tf.zeros((1, 96)), tf.zeros((1, 96))), (tf.zeros((1, 96)), tf.zeros((1, 96))),(tf.zeros((1, 1000)), tf.zeros((1, 1000)))])

In [3]:
## representation error
error_z = tf.squared_difference(z_n,z_s)
sumerror_z = tf.reduce_sum(error_z)

In [12]:
z_params = tf.trainable_variables(scope='encoder')
z_optimizer = tf.train.GradientDescentOptimizer(.1)
z_gradients = tf.gradients(sumerror_z, z_params)
clipped_gradients, norm = tf.clip_by_global_norm(z_gradients, clip_norm=5.)
update_z = z_optimizer.apply_gradients(grads_and_vars=zip(clipped_gradients, z_params))